# Fake News Classification - Text Pre-Processing

# Imports and Constants

In [1]:
import pandas as pd

In [2]:
DATA_PATH = '../data/'
CLEAN_DATA_FILE_NAME = 'news_dataset_cleaned.csv'

# Load Data

In [3]:
df = pd.read_csv(DATA_PATH + CLEAN_DATA_FILE_NAME)

In [4]:
df.head()

,title,text,subject,date,label,title_len,text_len,caps_in_title,norm_caps_in_title,caps_in_text,norm_caps_in_text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,2017-12-31,fake,79,2893,11,0.139241,138,0.047701
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,2017-12-31,fake,69,1898,8,0.115942,88,0.046365
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,2017-12-30,fake,90,3597,15,0.166667,308,0.085627
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,2017-12-29,fake,78,2774,19,0.243590,123,0.044340
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,2017-12-25,fake,70,2346,11,0.157143,63,0.026854


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38638 entries, 0 to 38637
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               38638 non-null  object 
 1   text                38638 non-null  object 
 2   subject             38638 non-null  object 
 3   date                38638 non-null  object 
 4   label               38638 non-null  object 
 5   title_len           38638 non-null  int64  
 6   text_len            38638 non-null  int64  
 7   caps_in_title       38638 non-null  int64  
 8   norm_caps_in_title  38638 non-null  float64
 9   caps_in_text        38638 non-null  int64  
 10  norm_caps_in_text   38638 non-null  float64
dtypes: float64(2), int64(4), object(5)
memory usage: 3.2+ MB


# Clean Title and Text Data